In [1]:
import pandas as pd
import os
import random
import json
from ast import literal_eval
import re
from utils import *
from collections import Counter

In [2]:
similar_intent_map = {'express concerns about a player from a failed quest team and suggest to not include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'question the leader why they selected a particular player for the team': 'question the leader why they selected a particular player for the team',
         'counter the team proposal because you think a player on the team is evil': 'counter the team proposal citing that you think a player on the team is evil',
         'counter the team proposal because a player on the team is evil': 'counter the team proposal citing that you think a player on the team is evil',
         'stay hidden in discussions and act like a loyal servant to protect yourself': 'stay hidden in discussions and act like a loyal servant to protect yourself',
         'stay hidden in discussions': 'stay hidden in discussions and act like a loyal servant to protect yourself',
         'stay hidden in discussions to protect yourself': 'stay hidden in discussions and act like a loyal servant to protect yourself',
         'stay hidden as loyal servant': 'stay hidden in discussions and act like a loyal servant to protect yourself',
         'support team proposal': 'support team proposal',
         'support the team proposal': 'support team proposal',
         'question a player on why he didnt vote for the last team': 'question a player on why he didnt vote for the last team',
         'question a player on why they didnt vote for the last team': 'question a player on why he didnt vote for the last team',
         'defend yourself against allegations that you could be evil': 'defend yourself against allegations that you could be evil',
         'support your teammate by expressing that he is good': 'support your teammate by expressing that he is good',
         'question a player about why they changed their statements': 'question a player about why they changed their statements',
         'provide hints to loyal players about evil players': 'provide hints to loyal players about evil players',
         'provide hints to loyal players': 'provide hints to loyal players about evil players',
         'guide team subtly': 'provide hints to loyal players about evil players',
         'provide hints about evil players': 'provide hints to loyal players about evil players',
         'express concerns about a player from a failed quest team and suggest to not include them in current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest to not include them in current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest not including them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest not to include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest to not include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest and suggest to not include them in current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about including a player from a failed quest': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'pretend to be merlin by providing hints on who is evil': 'pretend to be merlin by providing hints on who is evil',
         'express disagreement and vote disagree only if you are not in the proposed team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree if not on team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree if not in the team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree if not in team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree if not on the team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement if not in team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'counter the team proposal because you are good and not part of the team': 'counter the team proposal citing that you are good and not part of the team',
         'express confusion for failure of mission': 'express confusion for failure of mission',
         'express confusion for failure': 'express confusion for failure of mission',
         'express confusion for failed mission': 'express confusion for failure of mission',
         'include loyal players on the quest team as leader': 'include loyal players on the quest team as leader',
         'include loyal players on quest team as leader': 'include loyal players on the quest team as leader',
         'pretend to have information and act like merlin': 'pretend to have information and act like merlin',
         'pretend to be merlin to mislead': 'pretend to have information and act like merlin',
         'support your teammate to be on the quest team': 'support your teammate to be on the quest team',
         'support teammate to be on the quest team': 'support your teammate to be on the quest team',
         'support teammate on quest team': 'support your teammate to be on the quest team',
         'support teammate on team': 'support your teammate to be on the quest team',
         'cast suspicion on innocent players': 'cast suspicion on innocent players',
         'cast suspicion on loyal players': 'cast suspicion on innocent players',
         'support one loyal player': 'support one loyal player',
         'support loyal players': 'support one loyal player',
         'protect loyal players': 'support one loyal player',
         'express trust in loyal players': 'support one loyal player',
         'share concerns about an evil player': 'share concerns about an evil player',
         'express concerns about an evil player': 'share concerns about an evil player',
         'support loyal players of the previous quest team if the mission failed': 'support loyal players of the previous quest team if the mission failed',
         'defend previous quest team members': 'support loyal players of the previous quest team if the mission failed',
         'support loyal players of the previous quest team': 'support loyal players of the previous quest team if the mission failed',
         'defend your teammate against allegations that they could be evil': 'defend your teammate against allegations that they could be evil',
         'defend your teammate against allegations': 'defend your teammate against allegations that they could be evil',
         'defend teammate against allegations': 'defend your teammate against allegations that they could be evil',
         'defend teammate': 'defend your teammate against allegations that they could be evil',
         'defend teammate against suspicion': 'defend your teammate against allegations that they could be evil',
         'question the leader why they didnt put themselves on the quest team': 'question the leader why they didnt put themselves on the quest team',
         'strongly convince the team leader to choose you on team': 'strongly convince the team leader to choose you on team',
         'reveal identity by telling others you are servant to persuade other loyal players': 'reveal identity by telling others you are servant to persuade other loyal players',
         'defend the proposed team composition if it includes loyal players': 'defend the proposed team composition if it includes loyal players',
         'defend proposed team if includes loyal players': 'defend the proposed team composition if it includes loyal players',
         'defend the proposed team composition if includes loyal players': 'defend the proposed team composition if it includes loyal players',
         'support loyal players on team': 'defend the proposed team composition if it includes loyal players',
         'support loyal players on the quest team': 'defend the proposed team composition if it includes loyal players',
         'support the leaders decision': 'support team proposal',
         'express you are a loyal player': 'express you are a loyal player',
         'question why a player didnt vote for the last team': 'question a player on why he didnt vote for the last team',
         'support a loyal player': 'support one loyal player',
         'counter the team proposal because i suspect a player on the team is evil': 'counter the team proposal citing that you think a player on the team is evil',
         'counter the team proposal because you think a player on the team is evil': 'counter the team proposal citing that you think a player on the team is evil',
         'express concerns about a player from a failed quest team and suggest not including them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'strongly convince the team leader to choose you on the team': 'strongly convince the team leader to choose you on team',
         'question the leader why they selected player for the team': 'question the leader why they selected a particular player for the team',
         'express concerns about including player in the quest team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'counter previous players suspicion on me by expressing loyalty': 'defend yourself against allegations that you could be evil',
         'defend yourself against allegations of being evil': 'defend yourself against allegations of being evil',
         'defend yourself against allegations': 'defend yourself against allegations',
         'question player why they didnt include themselves in the proposed team': 'question the leader why they didnt put themselves on the quest team',
         'question player about their sudden change in stance': 'question a player about why they changed their statements',
         'express loyalty to the team': 'express you are a loyal player',
         'question a player on why they changed their statements': 'question a player about why they changed their statements',
        'pretend that you dont have enough information about who is evil right now': 'pretend that you dont have enough information about who is evil right now',
        'express that you dont have any information on whom to put on the team to protect yourself': 'express that you dont have any information on whom to put on the team to protect yourself',
         'reveal identity by telling others you are percival to persuade other loyal players': 'reveal identity by telling others you are percival to persuade other loyal players',
         'defend the team composition': 'support team proposal',
         'pretend to be percival by telling others you are percival': 'pretend to be percival by telling others you are percival',
         'pretend to be percival': 'pretend to be percival by telling others you are percival',
        'support the proposed team': 'support team proposal',
        'express concerns about a player from a failed quest and suggest to not include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
        'express disagreement if not on the team': 'express disagreement and vote disagree only if you are not in the proposed team',
        'stay hidden as servant': 'stay hidden in discussions and act like a loyal servant to protect yourself',
        'stay hidden in discussions as a loyal servant': 'stay hidden in discussions and act like a loyal servant to protect yourself',
        'express disagreement and vote disagree': 'express disagreement and vote disagree only if you are not in the proposed team',
        'express disagreement and vote disagree if not on the proposed team': 'express disagreement and vote disagree only if you are not in the proposed team',
        'express concerns about including a player from a failed quest team and suggest to not include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
        'express concerns about a player from a failed quest team and suggest not including them': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
        'express disagreement and vote disagree if not in the proposed team': 'express disagreement and vote disagree only if you are not in the proposed team',
        'stay hidden in discussions to protect self': 'stay hidden in discussions and act like a loyal servant to protect yourself',
        'stay hidden': 'stay hidden in discussions and act like a loyal servant to protect yourself',
        'stay hidden as evil': 'stay hidden in discussions and act like a loyal servant to protect yourself',

}

In [3]:
print(len(similar_intent_map))
uniq_intents = set(similar_intent_map.values())
for intent in uniq_intents:
    if intent not in similar_intent_map:
        similar_intent_map[intent] = intent
print(len(similar_intent_map))

102
105


In [4]:
intent_stats_dic = {}
def calc_intent_stats(summarized_intents, selected_intents):
    for intent in summarized_intents:
        if intent not in intent_stats_dic:
            intent_stats_dic[intent] = {'count': 0, 'summary': 0, 'selection': 0, 'both': 0}
        intent_stats_dic[intent]['count'] += 1
        intent_stats_dic[intent]['summary'] += 1
        if intent in selected_intents:
            intent_stats_dic[intent]['selection'] += 1
            intent_stats_dic[intent]['both'] += 1

    for intent in selected_intents:
        if intent not in intent_stats_dic:
            intent_stats_dic[intent] = {'count': 0, 'summary': 0, 'selection': 0, 'both': 0}
        if intent not in summarized_intents:
            intent_stats_dic[intent]['count'] += 1
            intent_stats_dic[intent]['selection'] += 1

In [87]:
# output/gpt-3_5-turbo-games/annotated/evaluation/summarization_annotations/complete_intent_following_without_intent_100_deepayan.csv
# output/gpt-3_5-turbo-games/annotated/evaluation/summarization_annotations/complete_intent_following_without_intent_100_javin_2.csv
# output/gpt-3_5-turbo-games/annotated/evaluation/summarization_annotations/complete_intent_following_without_intent_100_sy.csv

# gpt 4
# output/gpt-4-games/annotated/evaluation/summarization_annotations/intent_summarization_gpt4_without_intent_50_javin.csv
# output/gpt-4-games/annotated/evaluation/summarization_annotations/intent_summarization_gpt4_without_intent_50_Ziyi.csv
# ANNOTATED_FILE = "output/gpt-4-games/annotated/evaluation/summarization_annotations/intent_summarization_gpt4_without_intent_50_Ziyi.csv"
ANNOTATED_FILE = "output/gpt-4-games/annotated/evaluation/summarization_annotations/intent_summarization_gpt4_without_intent_50_Abhi.csv"

In [88]:
summarization_annotated_df = pd.read_csv(ANNOTATED_FILE)
summarization_annotated_df.head()

,id,context,think,speak,summarized_intents
0,game_2024-03-29-0-16-48-player4-5_form,**Name**: Player4\n\n**Role**: Percival\n\n**R...,I need to focus on protecting Merlin's identit...,"Player5, your selection intrigues me; could yo...",Question a player about why they changed their...
1,game_2024-03-29-1-59-14-player1-2_form,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,"As Merlin, I must use the information I have a...","I find the team selection intriguing, Player2....",Question the leader why they selected a partic...
2,game_2024-03-29-0-16-48-player1-3_form,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,"As Merlin, I must cast doubt on the evil playe...","Player3, as the leader, could you share the in...",Question the leader why they selected a partic...
3,game_2024-03-29-0-51-34-player2-2_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,"As Morgana, my first sub-goal is to emulate Me...","Fellow knights, based on our discussions and t...",Pretend to have information and act like Merli...
4,game_2024-03-29-1-59-14-player2-3_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,I need to subtly undermine the trust in the gr...,"Player3, your choice of including Player4 is i...",Question the leader why they selected a partic...


In [89]:
def clean_annotation(annotation):
    # sy
    split_intents = annotation.split('\n')
    summarized_intents = []
    for line in split_intents:
        summarized_intents.extend([remove_non_alphabets(val.split('. ')[-1]) for val in line.split(",") if val.strip() != ""])
    summarized_intents_final = []
    for intent in summarized_intents:
        split_intents = intent.strip().split("  ")
        split_intents = [i.strip() for i in split_intents]
        summarized_intents_final.extend(split_intents)

    return ','.join(summarized_intents_final)

In [90]:
def clean_annotation_v2(annotation):
    # javin
    summarized_intents = [remove_non_alphabets(intent.split('. ')[-1]) for intent in annotation.split('\n')]
    summarized_intents_final = []
    for intent in summarized_intents:
        split_intents = intent.strip().split("  ")
        split_intents = [i.strip() for i in split_intents]
        summarized_intents_final.extend(split_intents)
    return ','.join(summarized_intents_final)

In [91]:
def clean_annotation_v3(annotation):
    # deepayan
    summarized_intents = [remove_non_alphabets(intent).strip() for intent in annotation.split('\n')]
    return ','.join(summarized_intents)

In [92]:
def clean_annotation_v4(annotation):
    # just comma separated
    if (type(annotation) != str):
        return ''
    split_intents = re.split(',', annotation)
    sum_intents = []
    for line in split_intents:
        if len(line) > 10:
            processed_intent = remove_non_alphabets(line.strip())
            sum_intents.append(processed_intent)

    # summarized_intents = [remove_non_alphabets(intent.split('. ')[-1]) for intent in r[1].split('\n')]
    # guessed_intents_final = []
    # for intent in guessed_intents:
    #     split_intents = intent.strip().split("  ")
    #     split_intents = [i.strip() for i in split_intents]
    #     guessed_intents_final.extend(split_intents)

    return ','.join(sum_intents)

In [93]:
# sy
# summarization_annotated_df['summarized_intents'] = summarization_annotated_df['intention'].apply(lambda x: clean_annotation(x))
# deepayan
# summarization_annotated_df['summarized_intents'] = summarization_annotated_df['intentions_summary'].apply(lambda x: clean_annotation_v3(x))
# javin
# summarization_annotated_df['summarized_intents'] = summarization_annotated_df['corrected'].apply(lambda x: clean_annotation(x))

# gpt 4
# javin
summarization_annotated_df['summarized_intents'] = summarization_annotated_df['summarized_intents'].apply(lambda x: clean_annotation_v4(x))
summarization_annotated_df.head()

,id,context,think,speak,summarized_intents
0,game_2024-03-29-0-16-48-player4-5_form,**Name**: Player4\n\n**Role**: Percival\n\n**R...,I need to focus on protecting Merlin's identit...,"Player5, your selection intrigues me; could yo...",question a player about why they changed their...
1,game_2024-03-29-1-59-14-player1-2_form,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,"As Merlin, I must use the information I have a...","I find the team selection intriguing, Player2....",question the leader why they selected a partic...
2,game_2024-03-29-0-16-48-player1-3_form,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,"As Merlin, I must cast doubt on the evil playe...","Player3, as the leader, could you share the in...",question the leader why they selected a partic...
3,game_2024-03-29-0-51-34-player2-2_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,"As Morgana, my first sub-goal is to emulate Me...","Fellow knights, based on our discussions and t...",pretend to have information and act like merli...
4,game_2024-03-29-1-59-14-player2-3_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,I need to subtly undermine the trust in the gr...,"Player3, your choice of including Player4 is i...",question the leader why they selected a partic...


In [94]:
summarization_annotated_df = summarization_annotated_df.dropna(subset=["id"])
# summarization_annotated_df.info()

In [95]:
summarization_annotated_df['summarized_intents'].values[:5]

array(['question a player about why they changed their statements,question the leader why they selected a particular player for the team,express concerns about a player from a failed quest team and suggest to not include them in current team',
       'question the leader why they selected a particular player for the team,counter the team proposal because you think a player on the team is evil',
       'question the leader why they selected a particular player for the team,express concerns about a player from a failed quest team and suggest to not include them in current team,provide hints to loyal players about evil players',
       'pretend to have information and act like merlin,support your teammate to be on the quest team,cast suspicion on innocent players',
       'question the leader why they selected a particular player for the team,express concerns about a player from a failed quest team and suggest to not include them in current team,cast suspicion on innocent players'],
     

In [96]:
# all_samples_df = pd.read_csv("output/gpt-3_5-turbo-games/annotated/evaluation/structured_context_files/complete_intent_following.csv")

# gpt 4
all_samples_df = pd.read_csv("output/gpt-4-games/annotated/evaluation/structured_context_files/intent_summarization_gpt4_50.csv")

all_samples_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       50 non-null     object
 1   context  50 non-null     object
 2   intents  50 non-null     object
 3   think    50 non-null     object
 4   speak    50 non-null     object
dtypes: object(5)
memory usage: 2.1+ KB


In [97]:
annotation_results = summarization_annotated_df[['id', 'summarized_intents']].values.tolist()
len(annotation_results)

50

In [98]:
all_data = all_samples_df[['id', 'intents']].values.tolist()
len(all_data)

50

In [99]:
all_data_dic = {key: literal_eval(intents) for key, intents in all_data}
len(all_data_dic)

50

In [100]:
# all_data_dic['game_2024-03-06-16-53-2-player1-2_form']

In [101]:
annotated_samples = []
missing_count = 0
for r in annotation_results:
    if r[0] not in all_data_dic:
        missing_count += 1
        continue
    data = {}
    data['id'] = r[0]
    data['intents'] = all_data_dic[r[0]]
    split_intents = r[1].split('\n')
    data['summarized_intents'] = r[1].split(',')
    annotated_samples.append(data)

In [102]:
missing_count

0

In [103]:
results = []
for sample in annotated_samples:
    if 'summarized_intents' not in sample:
        print('not found')
        continue
    intents = [remove_non_alphabets(intent) for intent in sample['intents']]
    intents = [similar_intent_map[intent] for intent in intents if intent in similar_intent_map]
    summarized_intents = [similar_intent_map[remove_non_alphabets(intent)] for intent in sample['summarized_intents'] if remove_non_alphabets(intent) in similar_intent_map]
    
    results.append({'summarized_intents': summarized_intents, 'intents': intents})

In [104]:
results[0]

{'summarized_intents': ['question a player about why they changed their statements',
  'question the leader why they selected a particular player for the team',
  'express concerns about a player from a failed quest team and suggest to remove them from current team'],
 'intents': ['question a player about why they changed their statements',
  'question the leader why they selected a particular player for the team',
  'express concerns about a player from a failed quest team and suggest to remove them from current team']}

In [105]:
count_sum, count_sel = 0, 0
for r in results:
    count_sum += len(r['summarized_intents'])
    count_sel += len(r['intents'])
count_sum, count_sel

(142, 150)

In [106]:
intent_stats_dic = {}
for res in results:
    calc_intent_stats(res['summarized_intents'], res['intents'])

In [107]:
total_selection_count = 0
selection_in_summary_count = 0
total_summary_count = 0
for key, value in intent_stats_dic.items():
    total_selection_count += value['selection']
    selection_in_summary_count += value['both']
    total_summary_count += value['summary']

In [108]:
recall = selection_in_summary_count / total_selection_count
recall

0.8466666666666667

In [109]:
precision = selection_in_summary_count/total_summary_count
precision

0.8943661971830986

In [110]:
calculate_f1_score(recall, precision)

0.8698630136986302

Javin gpt 3.5 summarization 100 samples => f1 = 0.7659, recall = 0.72262, precision = 0.8148 <br/>
Sy gpt 3.5 summarization 100 samples => f1 = 0.7442, recall = 0.6763, precision = 0.8274 <br/>
Deepayan gpt 3.5 summarization 100 samples => f1 = 0.7462, recall = 0.7261, precision = 0.7675 <br/>

std = 0.0097

GPT 4 <br/>
Javin summmarization 50 samples => f1 = 0.8273, recall = 0.7666, precision = 0.8984 <br/>
Ziyi summarization 50 samples => f1 = 0.8041, recall = 0.78, precision = 0.8297<br/>
Abhishek summarization 50 samples => f1 = 0.8698, recall = 0.8466, precision = 0.894

std = 0.0272

# Model results

In [5]:
# RESULTS_FILE = "output/gpt-3_5-turbo-games/annotated/evaluation/summarization_model_results/intent_summarization_model_results_n5.json"
RESULTS_FILE = "output/gpt-4-games/annotated/evaluation/summarization_model_results/intent_summarization_all_results.json"

In [6]:
with open(RESULTS_FILE) as f:
    data = json.load(f)
samples = list(data.values())

In [7]:
samples[1]

{'id': 'game_2024-03-29-12-52-20-player1-2_ref',
 'context': "**Name**: Player1\n\n**Role**: Assassin\n\n**Role Details**: \nYou are assassin. These are your goals and characteristics:\n- Discreetly seek Merlin's identity.\n- Work to prevent Quests success.\n- Goal(Main Intent): Win either by failing quests or assassinating Merlin.\n- Morgana is Player2\n\n**Round**: 2\n\n**Current Leader**: Player2\n\n**Current Team**: Player2, Player3, Player5\n\n**Previous Rounds Team Voting**:\nRound 1: Player1 = agree, Player2 = agree, Player3 = agree, Player4 = agree, Player5 = disagree \nRound 2: Player1 = agree, Player2 = agree, Player3 = agree, Player4 = agree, Player5 = disagree \n\n**Previous Results**: \nRound 1: Team = Player1, Player2, Result = failure\n\n**Previous Rounds Summary**:\nSummary questions: \n\nWho was the leader for this round?\nWhat was the proposed team for this round?\nWhy do you think the leader proposed the current team?\nDid everyone agree for the proposed team?\nIf so

In [8]:
samples_list = []
total_sum_intents = 0
total_sel_intents = 0
for s in samples:
    if 'summarized_intents' not in s:
        continue
    total_sum_intents += len(s['summarized_intents'])
    total_sel_intents += len(s['intents'])
    samples_list.append([s['id'], s['summarized_intents'], s['intents']])

In [9]:
total_sel_intents, total_sum_intents

(270, 250)

## Cleaning rephrased intents

In [10]:
uniq_sum_intents = []
for ann in samples_list:
    for intent in ann[1]:
        intent = remove_non_alphabets(intent)
        uniq_sum_intents.append(intent)

print(len(uniq_sum_intents))
uniq_sum_intents1 = set(uniq_sum_intents)
print(len(uniq_sum_intents1))

count = 0
for intent in uniq_sum_intents:
    if intent not in similar_intent_map.keys():
        print(intent)
        count += 1
count

250
21


0

## Results

In [11]:
annotated_samples = []
missing_count = 0
for r in samples_list:
    data = {}
    data['intents'] = r[2]
    # split_intents = r[1].split('\n')
    
    data['summarized_intents'] = [similar_intent_map[remove_non_alphabets(intent)] for intent in r[1] if remove_non_alphabets(intent) in similar_intent_map]
    if len(data['summarized_intents']) != len(r[1]):
        print(len(data['summarized_intents']) - len(r[1]))
    annotated_samples.append(data)

In [12]:
len(annotated_samples)

91

In [13]:
annotated_samples[1]

{'intents': ['defend your teammate against allegations that they could be evil',
  'express concerns about a player from a failed quest team and suggest to not include them in current team',
  'support team proposal'],
 'summarized_intents': ['support team proposal',
  'defend your teammate against allegations that they could be evil',
  'cast suspicion on innocent players']}

In [14]:
results = []
for sample in annotated_samples:
    intents = [similar_intent_map[remove_non_alphabets(intent)] for intent in sample['intents'] if remove_non_alphabets(intent) in similar_intent_map]
    summarized_intents = [remove_non_alphabets(intent) for intent in sample['summarized_intents']]
    
    results.append({'summarized_intents': summarized_intents, 'intents': intents})

In [15]:
intent_stats_dic = {}
for res in results:
    calc_intent_stats(res['summarized_intents'], res['intents'])

In [16]:
total_selection_count = 0
selection_in_summary_count = 0
total_summary_count = 0
for key, value in intent_stats_dic.items():
    total_selection_count += value['selection']
    selection_in_summary_count += value['both']
    total_summary_count += value['summary']

In [17]:
recall = selection_in_summary_count / total_selection_count
recall

0.8199233716475096

In [18]:
precision = selection_in_summary_count/total_summary_count
precision

0.856

In [19]:
total_summary_count

250

In [20]:
total_selection_count

261

In [21]:
calculate_f1_score(recall, precision)

0.837573385518591

GPT 4 - all f1 = 0.8616, recall = 0.8320, precision = 0.8934 <br/>
GPT 3.5 - all f1 = 0.6552, recall = 0.5910, precision = 0.7350

GPT 3.5 all summarization (without intent fix) = f1 = 0.6949, recall = 0.6360, precision = 0.7659<br/>
GPT 4 all summarization = f1 = 0.83757, recall = 0.8199, precision = 0.8560<br/>